<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [970]:
# ##### PORTFOLIO INVESTOR CODE #####
#
# Code to upload data in Google Sheets to support investiments decisions.
# The lists with the tickers and other informations should be filled directly in the code where there are INSERT OR ADJUST HERE lines.
# IFIX historic series csv file with time desired (2y, 3y, etc.) from Investing.com should be downloaded to a Google Drive directory which is in the path in code - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data .
# FUNDS EXPLORER - Copy and paste date from https://www.fundsexplorer.com.br/ranking in a Google Drive directory excel file which is in the path in code. It is necessary to SELECT ALL COLUMNS in Funds Explorer website.
# Before to run the code it is necessary to make these activities. Check the code below until the ENDING POINT TO MANUAL ACTIVITIES.
# Verifiy data on thes files in case of code running problems.

In [971]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
import os
import datetime as dt
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

In [972]:
# Stock tickers - Insert here# Real Estate Invesments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IBOV + 25 assets in alphabetical order to easly adjust google docs spreadsheet
tickers = ['^BVSP','ABEV3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CMIN3.SA','CPFE3.SA','CPLE6.SA','ELET6.SA','FLRY3.SA','ITUB4.SA','IVVB11.SA','JHSF3.SA','KLBN11.SA','MBRF3.SA','LEVE3.SA','LREN3.SA','NEOE3.SA','PETR4.SA','POMO4.SA','QQQI11.SA','RECV3.SA','RENT3.SA','SBSP3.SA','TGMA3.SA','TIMS3.SA','TUPY3.SA','VALE3.SA','VIVA3.SA','VIVT3.SA','WEGE3.SA']

In [973]:
# Real Estate Investments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IFIX + 20 assets in alphabetical order to easly adjust google docs spreadsheet
# IFIX here only to a space in dataframe, yahoo finance return IFIX.SA maximum 5 days. IFIX should be calculated downloading an Investing.com csv file history and it must be uploaded in personal Google Drive. This code read it.
tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MXRF11.SA','MFII11.SA','SAPI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']

In [974]:
# US Stocks tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 20 assets in alphabetical order to easly adjust google docs spreadsheet
tickersus = ['^GSPC','USDBRL=X','AAPL','AMZN','ASML','BAC','BKNG','BMY','BRK-B','CRWD','EXC','GOOG','HALO','JNJ','JPM','KMB','KO','LLY','META','MRK','MSFT','NOW','NVDA','PFE','PLTR','RIO','T','TSLA','TSM','V','XOM','WMT']

In [975]:
# US ETFs tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 10 assets in alphabetical order to easly adjust google docs spreadsheet
tickereus = ['^GSPC','USDBRL=X','FBTC','JEPI','HACK', 'IVV','SCHD','SOXX','SPY','TLT','SMH','TFLO']

In [976]:
# US ETFs REITS tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 10 assets in alphabetical order to easly adjust google docs spreadsheet
tickerrus = ['USRT','USDBRL=X','O','STAG','ADC', 'LTC','SLG','WELL','PLD','AMT','EQIX','SPG']

In [977]:
# Portfolio tickers - INSERT OR ADJUST HERE
tickerport = ['^BVSP','USDBRL=X','AMZN','BBSE3.SA','BRK-B','CPFE3.SA','ELET6.SA','FBTC','HTMX11.SA','JHSF3.SA','JSAF11.SA','KMB','MRK','MSFT','PETR4.SA','POMO4.SA','SAPI11.SA','SCHD','TFLO','TGAR11.SA','TRXF11.SA','VALE3.SA']
# Portfolio tickers weight - INSERT OR ADJUST HERE IN THE SAME ORDER!
weightport = [0.000, 0.000, 5.081, 7.509, 10.812, 0.000, 8.292, 2.714, 5.311, 0.000, 6.269, 3.388, 3.388, 3.388, 13.885, 0.000, 6.410, 1.355, 3.389, 4.908, 13.882, 0.000]
# Portfolio tickers expected returns - INSERT OR ADJUST HERE IN THE SAME ORDER!
# IBOV (^BVSP) and USD (USDBRL=X) MUST always be zero!
expretport = [0.0, 8.0, 23.9, 18.5, 18.5, 2.7, 6.0, 25.9, 20.5, 25.3, 28.1, 19.8, 24.6, 21.5, 27.4, 13.8, 17.1, 8.6, 8.6, 22.1, 19.4, 5.8]
# Portfolio assets limits - INSERT OR ADJUST HERE IN THE SAME ORDER!
# IBOV (^BVSP) and USD (USDBRL=X) MUST always be zero!
limitsassetsport = [0.0, 0.0, 10.0, 10.0, 10.0, 10.0, 5.0, 5.0, 10.0, 5.0, 6.0, 5.0, 5.0, 5.0, 15.0, 5.0, 6.0, 5.0, 5.0, 5.0, 15.0, 10.0]
fatorcorr = 0.90987 # FATOR DE CORREÇÃO para os pesos de acordo com quanto do total do portfolio é renda variável
limitsassetsport = [x * fatorcorr for x in limitsassetsport]
# Print just to verify lists lengths
print(len(tickerport))
print(len(weightport))
print(len(expretport))
print(len(limitsassetsport))


22
22
22
22


In [978]:
# Risk free rate in percentage - INSERT OR ADJUST HERE
riskfree = 13.88

In [979]:
# Target Sharpe in percentage of Sharpe Estimated Maximum - INSERT OR ADJUST HERE
target_per = 85.0

In [980]:
# Portfolio dataframe creation
portfolio = pd.DataFrame({'Ticker': tickerport, 'W': weightport, 'RetE%':expretport})

In [981]:
# Excluding .SA, renaming ^BVSP to IBOV and USDBRL=X to USDBRL
portfolio['Ticker'] = portfolio['Ticker'].str.replace('.SA', '', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('^BVSP', 'IBOV', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('USDBRL=X', 'USDBRL', regex=False)
# display(portfolio)

In [982]:
# Load tickers history prices in a dataframe considering a certain period of time - ADJUST HERE, default 1 year (1y)
# Sometimes some ticker has problems in yahoo finance. If it happens, close the session and try again. Or change the ticker, because problem in one ticker will cause problem in all code running.
# Check success download completed to all dataframes, otherwise the code will broke in next lines.
dfs = yf.download(tickers, period='2y', auto_adjust=True)['Close']
dfr = yf.download(tickerr, period='2y', auto_adjust=True)['Close']
dfsus = yf.download(tickersus, period='2y', auto_adjust=True)['Close']
dfeus = yf.download(tickereus, period='2y', auto_adjust=True)['Close']
dfrus = yf.download(tickerrus, period='2y', auto_adjust=True)['Close']
dfport = yf.download(tickerport, period='2y', auto_adjust=True)['Close']
# Remove timezone from index
dfs.index = pd.to_datetime(dfs.index).tz_localize(None)
dfr.index = pd.to_datetime(dfr.index).tz_localize(None)
dfsus.index = pd.to_datetime(dfsus.index).tz_localize(None)
dfeus.index = pd.to_datetime(dfeus.index).tz_localize(None)
dfrus.index = pd.to_datetime(dfrus.index).tz_localize(None)
dfport.index = pd.to_datetime(dfport.index).tz_localize(None)
# display(dfs)
# display(dfr)
# display(dfeus)
# display(dfrus)
# display(dfport)

[*********************100%***********************]  31 of 31 completed
[*********************100%***********************]  21 of 21 completed
[*********************100%***********************]  32 of 32 completed
[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  22 of 22 completed


In [983]:
# IFIX historic series from Investing.com to be appended in real state dataframe dfr - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data - MAKE THIS ACTIVITY
# Download the file from site and copy to your google drive. Rename de file as history.csv. Adjust the path below in " ifixfile = .... " command line according your file location.
# Google Drive mounth
drive.mount('/content/drive', force_remount=True)
# File path on Google Drive - Download the file and upload to Financas folder in Google Drive. Rename the path according file name uploaded.
ifixfile = '/content/drive/MyDrive/Financas/history.csv'
# File csv to dataframe converting quote to float
ifix = pd.read_csv(ifixfile, thousands = '.', decimal = ',', dtype = {'Último':np.float64})
# Excluding and rename columns
ifix = ifix.drop(columns=['Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'])
ifix = ifix.rename(columns={'Data': 'Date', 'Último': 'IFIX.SA'})
# Date format in Date column
ifix['Date'] = pd.to_datetime(ifix['Date'], format='%d%m%Y', errors='coerce')
ifix.set_index('Date', inplace=True)
# Solve eventual duplicated registers, grouped by mean
ifix = ifix.groupby(level=0).mean()
# display(ifix)

Mounted at /content/drive


In [984]:
# Go to Funds Explorer Table in https://www.fundsexplorer.com.br/ranking  - MAKE THIS ACTIVITY
# Manually select all columns in web page and paste all data AS VALUES ONLY in a NEW Excel File sheet.
# Save the file with the name fundsexplorer.xlsx in a folder called Financas (or in another preferable path, adjusting the path below)
fundsexplorerfile = '/content/drive/MyDrive/Financas/fundsexplorer.xlsx'
# Excel file reading
fundsexplorer = pd.read_excel(fundsexplorerfile)
#display(fundsexplorer)
# The sequence of FUNDS EXPLORER ROUTINE, see at the end of the code.

In [985]:
# ENDING POINT TO MANUAL ACTIVITIES. The code should run after this point. Permission to Google Drive access can be required during code running.

In [986]:
# Replace dfr dataframe by ifix values by index key (date)
dfr.update(ifix)
# display(dfr)

In [987]:
# Excluding .SA, renaming ^BVSP to IBOV, ^GSPC to SP500
dfs.columns = [col.replace('.SA', '') for col in dfs.columns]
dfs.columns = [col.replace('^BVSP', 'IBOV') for col in dfs.columns]
dfr.columns = [col.replace('.SA', '') for col in dfr.columns]
dfsus.columns = [col.replace('^GSPC', 'SP500') for col in dfsus.columns]
dfsus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfsus.columns]
dfeus.columns = [col.replace('^GSPC', 'SP500') for col in dfeus.columns]
dfeus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfeus.columns]
# dfrus.columns = [col.replace('^GSPC', 'SP500') for col in dfrus.columns]
dfrus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfrus.columns]
dfport.columns = [col.replace('.SA', '') for col in dfport.columns]
dfport.columns = [col.replace('^BVSP', 'IBOV') for col in dfport.columns]
dfport.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfport.columns]

In [988]:
# Other conformations such as ascending order and the market indexes in the first column
dfs = dfs[sorted(dfs.columns)]
dfr = dfr[sorted(dfr.columns)]
dfsus = dfsus[sorted(dfsus.columns)]
dfeus = dfeus[sorted(dfeus.columns)]
dfrus = dfrus[sorted(dfrus.columns)]
dfport = dfport[sorted(dfport.columns)]
dfs = dfs[['IBOV'] + [col for col in dfs.columns if col != 'IBOV']]
dfr = dfr[['IFIX'] + [col for col in dfr.columns if col != 'IFIX']]
dfsus = dfsus[['SP500', 'USDBRL'] + [col for col in dfsus.columns if col not in ['SP500', 'USDBRL']]]
dfeus = dfeus[['SP500', 'USDBRL'] + [col for col in dfeus.columns if col not in ['SP500', 'USDBRL']]]
dfrus = dfrus[['USRT', 'USDBRL'] + [col for col in dfrus.columns if col not in ['USRT', 'USDBRL']]]
dfport = dfport[['IBOV', 'USDBRL'] + [col for col in dfport.columns if col not in ['IBOV', 'USDBRL']]]
# Here dataframes should be ready for calculations. It will be made and uploaded in dfxvar dataframes later.
# display(dfs)
# display(dfr)
# display(dfsus)
# display(dfeus)
# display(dfrus)
# display(dfport)

In [989]:
# Timeline harmonization, same period of time
common_idx = (
    dfs.index
    .intersection(dfr.index)
    .intersection(dfsus.index)
    .intersection(dfeus.index)
    .intersection(dfrus.index)
    .intersection(dfport.index)
)

# Reindex e preencher (sem warnings)
dfs = dfs.reindex(common_idx).ffill()
dfr = dfr.reindex(common_idx).ffill()
dfsus = dfsus.reindex(common_idx).ffill()
dfeus = dfeus.reindex(common_idx).ffill()
dfrus = dfrus.reindex(common_idx).ffill()
dfport = dfport.reindex(common_idx).ffill()


In [990]:
# Portfolio daframe in BRL
# Multiply USA assets by USDBRL value (BRL quotation)
#
# Insira o código aqui
# --- Início do fragmento: converter colunas cotadas em USD para BRL (usando dataframes normalizados) ---
usd_sources = []
for name in ('dfsus', 'dfeus', 'dfrus'):
    if name in globals():
        try:
            df_src = globals()[name]
            # considera apenas colunas (tickers) do dataframe de referência
            usd_sources.extend(list(df_src.columns))
        except Exception:
            pass

usd_set = set(usd_sources)

# identifica colunas de dfport que correspondem a ativos em USD (ignora a coluna de câmbio)
cols_a_converter = [c for c in dfport.columns if c != 'USDBRL' and c in usd_set]

# realiza a conversão (multiplica o preço em USD pela cotação USDBRL linha a linha)
for col in cols_a_converter:
    # multiplicação alinhando índices; preserva NaNs onde existirem
    dfport[col] = dfport[col].multiply(dfport['USDBRL'], axis=0)

# Resultado na tela: quais colunas foram convertidas e visualização inicial do dfport
if cols_a_converter:
    print(f"Convertidas para BRL (multiplicadas por dfport['USDBRL']): {cols_a_converter}")
else:
    print("Nenhuma coluna de dfport foi identificada como cotada em USD (nenhuma conversão aplicada).")

# mostrar as primeiras linhas do dfport para conferência
display(dfport.head())
# --- Fim do fragmento ---

Convertidas para BRL (multiplicadas por dfport['USDBRL']): ['AMZN', 'BRK-B', 'FBTC', 'KMB', 'MRK', 'MSFT', 'SCHD', 'TFLO']


,IBOV,USDBRL,AMZN,BBSE3,BRK-B,CPFE3,ELET6,FBTC,HTMX11,JHSF3,...,MRK,MSFT,PETR4,POMO4,SAPI11,SCHD,TFLO,TGAR11,TRXF11,VALE3
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-21,125607.0,4.8518,NaN,26.341257,NaN,36.360001,37.179810,NaN,128.500198,3.718372,...,NaN,NaN,26.448515,5.243311,6.459789,NaN,NaN,91.885391,85.268219,64.311378
2023-11-22,126035.0,4.8993,718.776350,26.440313,1772.566716,36.439999,37.560661,NaN,125.868950,3.701236,...,467.961094,1824.004881,26.577971,5.165441,6.666932,108.374765,225.606524,91.717331,84.922401,63.700047
2023-11-23,126576.0,4.9055,719.685928,26.498095,1774.809817,36.680000,38.296982,NaN,127.471893,3.718372,...,468.553277,1826.313075,26.585526,5.347139,6.521166,108.511908,225.892018,91.625664,84.961693,63.300064
2023-11-24,125542.0,4.9025,719.392899,26.341257,1778.038719,36.410000,37.518345,NaN,127.902878,3.709803,...,469.141871,1823.167514,26.661119,5.381749,6.518096,108.854901,225.753882,91.740250,84.718056,62.763905
2023-11-27,125683.0,4.9009,724.009919,26.176157,1770.891147,36.980000,38.254665,NaN,133.240967,3.701236,...,466.962970,1828.270473,26.509937,5.347139,6.565663,108.273751,225.724706,91.671524,84.977425,62.389458


In [991]:
# Calculate daily variation
dfsvar = dfs.pct_change()
dfrvar = dfr.pct_change()
dfsusvar = dfsus.pct_change()
dfeusvar = dfeus.pct_change()
dfrusvar = dfrus.pct_change()
dfportvar = dfport.pct_change()
# display(dfsvar)
# display(dfrvar)
# display(dfsusvar)
# display(dfeusvar)
# display(dfeusvar)
# display(dfportvar)

In [992]:
# Market Percentage Historic Return calculation and column add in output dataframes
# CAGR – Compound Annual Growth Rate - calculation
#
def cagr_from_prices(series):
    s = series.dropna()
    if len(s) < 2:
        return np.nan
    total_return = s.iloc[-1] / s.iloc[0] - 1
    n_days = len(s) - 1
    # anualiza para 252 dias úteis por ano
    return (1 + total_return) ** (252 / n_days) - 1
#
# Stocks - Aplica por coluna (cada coluna = um ticker)
cagr_series = dfs.apply(cagr_from_prices)  # retorna fração, ex: 0.069 = 6.9%
stockvar = cagr_series.mul(100).to_frame(name='RetH%')  # converte para %
stockvar.index.name = 'Ticker'
stockvar['RetH%'] = stockvar['RetH%'].round(1)
# display(stockvar)
#
# Real State - Aplica por coluna (cada coluna = um ticker)
cagr_series = dfr.apply(cagr_from_prices)  # retorna fração, ex: 0.069 = 6.9%
realstatevar = cagr_series.mul(100).to_frame(name='RetH%')  # converte para %
realstatevar.index.name = 'Ticker'
realstatevar['RetH%'] = realstatevar['RetH%'].round(1)
# display(realstatevar)
#
# Stock US - Aplica por coluna (cada coluna = um ticker)
cagr_series = dfsus.apply(cagr_from_prices)  # retorna fração, ex: 0.069 = 6.9%
stockusvar = cagr_series.mul(100).to_frame(name='RetH%')  # converte para %
stockusvar.index.name = 'Ticker'
stockusvar['RetH%'] = stockusvar['RetH%'].round(1)
# display(stockusvar)
#
# ETF US - Aplica por coluna (cada coluna = um ticker)
cagr_series = dfeus.apply(cagr_from_prices)  # retorna fração, ex: 0.069 = 6.9%
etfusvar = cagr_series.mul(100).to_frame(name='RetH%')  # converte para %
etfusvar.index.name = 'Ticker'
etfusvar['RetH%'] = etfusvar['RetH%'].round(1)
# display(etfusvar)
#
# ETF REITS US - Aplica por coluna (cada coluna = um ticker)
cagr_series = dfrus.apply(cagr_from_prices)  # retorna fração, ex: 0.069 = 6.9%
reitusvar = cagr_series.mul(100).to_frame(name='RetH%')  # converte para %
reitusvar.index.name = 'Ticker'
reitusvar['RetH%'] = reitusvar['RetH%'].round(1)
# display(reitusvar)
#
# Portfolio - Aplica por coluna (cada coluna = um ticker)
cagr_series = dfport.apply(cagr_from_prices)  # retorna fração, ex: 0.069 = 6.9%
portvar = cagr_series.mul(100).to_frame(name='RetH%')  # converte para %
portvar.index.name = 'Ticker'
portvar['RetH%'] = portvar['RetH%'].round(1)
# display(portvar)

In [993]:
# Market return variance calculation
vars = dfsvar.var()*252
varr = dfrvar.var()*252
varsus = dfsusvar.var()*252
vareus = dfeusvar.var()*252
varrus = dfrusvar.var()*252
varport = dfportvar.var()*252
# display(vars)
# display(varr)
# display(varsus)
# display(vareus)
# display(varrus)
# display(varport)

In [994]:
# Market risk calculation, in percentage (%). Add column in output dataframes
stockvar['Risk%'] = dfsvar.std()*np.sqrt(252)*100
stockvar['Risk%'] = stockvar['Risk%'].round(0)
realstatevar['Risk%'] = dfrvar.std()*np.sqrt(252)*100
realstatevar['Risk%'] = realstatevar['Risk%'].round(0)
stockusvar['Risk%'] = dfsusvar.std()*np.sqrt(252)*100
stockusvar['Risk%'] = stockusvar['Risk%'].round(0)
etfusvar['Risk%'] = dfeusvar.std()*np.sqrt(252)*100
etfusvar['Risk%'] = etfusvar['Risk%'].round(0)
reitusvar['Risk%'] = dfrusvar.std()*np.sqrt(252)*100
reitusvar['Risk%'] = reitusvar['Risk%'].round(0)
portvar['Risk%'] = dfportvar.std()*np.sqrt(252)*100
portvar['Risk%'] = portvar['Risk%'].round(0)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [995]:
# Covariance calculation
covs = dfsvar.cov()*252
covr = dfrvar.cov()*252
covsus = dfsusvar.cov()*252
coveus = dfeusvar.cov()*252
covrus = dfrusvar.cov()*252
covport = dfportvar.cov()*252
# display(covs)
# display(covr)
# display(covsus)
# display(coveus)
# display(covrus)
# display(covport)

In [996]:
# Beta calculation
betas = covs['IBOV']/vars['IBOV']
betas = betas.round(3)
betas.name = 'Beta'
betar = covr['IFIX']/varr['IFIX']
betar = betar.round(3)
betar.name = 'Beta'
betasus = covsus['SP500']/varsus['SP500']
betasus = betasus.round(3)
betasus.name = 'Beta'
betaeus = coveus['SP500']/vareus['SP500']
betaeus = betaeus.round(3)
betaeus.name = 'Beta'
betarus = covrus['USRT']/varrus['USRT']
betarus = betarus.round(3)
betarus.name = 'Beta'
betaport = covport['IBOV']/varport['IBOV']
betaport = betaport.round(3)
betaport.name = 'Beta'
# display(betas)
# display(betar)
# display(betasus)
# display(betaeus)
# display(betarus)
# display(betaport)

In [997]:
# Adding Beta to column output dataframes
stockvar['Beta'] = betas
realstatevar['Beta'] = betar
stockusvar['Beta'] = betasus
etfusvar['Beta'] = betaeus
reitusvar['Beta'] = betarus
portvar['Beta'] = betaport
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [998]:
# Adding Min to column output dataframes
stockvar['Min'] = dfs.min()
stockvar['Min'] = stockvar['Min'].round(2)
realstatevar['Min'] = dfr.min()
realstatevar['Min'] = realstatevar['Min'].round(2)
stockusvar['Min'] = dfsus.min()
stockusvar['Min'] = stockusvar['Min'].round(2)
etfusvar['Min'] = dfeus.min()
etfusvar['Min'] = etfusvar['Min'].round(2)
reitusvar['Min'] = dfrus.min()
reitusvar['Min'] = reitusvar['Min'].round(2)
portvar['Min'] = dfport.min()
portvar['Min'] = portvar['Min'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [999]:
# Adding Max to column output dataframes
stockvar['Max'] = dfs.max()
stockvar['Max'] = stockvar['Max'].round(2)
realstatevar['Max'] = dfr.max()
realstatevar['Max'] = realstatevar['Max'].round(2)
stockusvar['Max'] = dfsus.max()
stockusvar['Max'] = stockusvar['Max'].round(2)
etfusvar['Max'] = dfeus.max()
etfusvar['Max'] = etfusvar['Max'].round(2)
reitusvar['Max'] = dfrus.max()
reitusvar['Max'] = reitusvar['Max'].round(2)
portvar['Max'] = dfport.max()
portvar['Max'] = portvar['Max'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [1000]:
# Organizing columns order and making Index column as index of dataframes
stockvar = stockvar.reset_index()
realstatevar = realstatevar.reset_index()
stockusvar = stockusvar.reset_index()
etfusvar = etfusvar.reset_index()
reitusvar = reitusvar.reset_index()
portvar = portvar.reset_index()
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(reitusvar)
# display(portvar)

In [1001]:
# FUNCTIONS TO CALCULATE PORTFOLIO PERFORMANCE, MAXIMUM SHARPE AND TARGET SHARPE
# --- Exemplo de uso ---
# Supondo que você tem df_returns (DataFrame) com retornos log ou simples, calculados diariamente/mensalmente:
# mu = df_returns.mean().values           # retorno médio por ativo (periodicidade compatível)
# cov = df_returns.cov().values           # matriz de covariância
# risk_free = 0.0                         # ajuste conforme sua periodicidade (ex: taxa anual/252 para dia)
#
# res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
# print("Max Sharpe:", portfolio_performance(w_max, mu, cov, risk_free)[2], "pesos:", w_max)
#
# target = 1.2
# res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree)
# print("Resultado objetivo:", info)
# print("Pesos:", w_tgt)

In [1002]:
# Portfolio dataframe assembling
portfolio = pd.merge(portfolio, portvar, on='Ticker', how='inner')
# portfolio = portfolio[sorted(portfolio.columns)]
# portfolio = portfolio[['IBOV', 'USDBRL'] + [col for col in portfolio.columns if col not in ['IBOV', 'USDBRL']]]
# firstsnames = ['IBOV','USDBRL']
firstsnames = ['IBOV', 'USDBRL']
portfolio.iloc[2:] = portfolio[~portfolio['Ticker'].isin(firstsnames)].sort_values(by='Ticker').values
#
# display(portfolio)

In [1003]:
# Portfolio PERFORMANCE calculation function
def portfolio_performance(weights, mu, cov, riskfree):
    """
    Retorna (retorno_portfolio, volatilidade_portfolio, sharpe_portfolio)
    expected_returns: vetor de retornos esperados por ativo (numpy array)
    cov: matriz de covariância (numpy array)
    weights: vetor de pesos
    riskfree: taxa livre de risco (mesma periodicidade)
    """
    w = np.array(weights)
    ret = float(np.dot(w, mu))
    vol = float(np.sqrt(w.T @ cov @ w))
    if vol == 0:
        sharpe = 0.0
    else:
        sharpe = (ret - riskfree) / vol
    return ret, vol, sharpe

In [1004]:
riskfree = riskfree / 100 # Risk Free is indicated in percentage

In [1005]:
weights = [values / 100 for values in weightport] # Expected Returns are indicated in percentage

In [1006]:
# Convert covariance dataframe in numpy matrix
cov = covport.values

In [1007]:
# Extracting expected returns
mu = portfolio['RetE%'].values / 100

In [1008]:
# Porfolio Expected Return, Expected Risk and Expected Sharpe calculations
portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp = portfolio_performance(weights, mu, cov, riskfree)
# print (portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp)

In [1009]:
# Portfolio adding columns with Total Return, Total Risk and Sharpe values in first register(in the same line of IBOV value index). Other registers being filled with zero.
portfolio['RetET%'] = [portfolioretexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetET%'] = portfolio['RetET%'].round(1)
portfolio['RiskET%'] = [portfolioriskexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RiskET%'] = portfolio['RiskET%'].round(0)
portfolio['SharpeE'] = [portfoliosharpeexp] + [0] * (len(portfolio) - 1)
portfolio['SharpeE'] = portfolio['SharpeE'].round(3)

In [1010]:
# Extracting real returns
mu = portfolio['RetH%'].values / 100

In [1011]:
# Porfolio Real (historic) Return, Risk and Sharpe calculations
portfoliorettotal, portfoliorisktotal, portfoliosharpe = portfolio_performance(weights, mu, cov, riskfree)
# print (portfoliorettotal, portfoliorisktotal, portfoliosharpe)

In [1012]:
portfolio['RetHT%'] = [portfoliorettotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetHT%'] = portfolio['RetHT%'].round(1)
portfolio['SharpeH'] = [portfoliosharpe] + [0] * (len(portfolio) - 1)
portfolio['SharpeH'] = portfolio['SharpeH'].round(3)

In [1013]:
# Calculate Total Beta and add in IBOV line
portfolio.at[0, 'Beta'] = (portfolio['W'] / 100 * portfolio['Beta']).sum()
portfolio['Beta'] = portfolio['Beta'].round(3)
# display(portfolio)

In [1014]:
# Portfolio MAXIMUM SHARPE calculation function
def max_sharpe_weights(mu, cov, risk_free, bounds=None, constraints=None):
    """
    Encontra os pesos que maximizam o Sharpe (usando minimize sobre -sharpe).
    bounds: lista de tuplas (min, max) por ativo. padrão: (0,1) para todos.
    constraints: lista de constraint dicts para scipy.optimize.minimize (opcional).
    Retorna o dicionário de resultado e os pesos ótimos.
    """
    n = len(mu)
    if bounds is None:
        bounds = tuple((0.0, 1.0) for _ in range(n))
    # restrição: soma dos pesos = 1
    cons = [{'type':'eq', 'fun': lambda w: np.sum(w) - 1.0}]
    if constraints:
        cons.extend(constraints)
    # inicialização: pesos iguais
    x0 = np.ones(n) / n

    def neg_sharpe(w):
        _, _, s = portfolio_performance(w, mu, cov, riskfree)
        return -s

    res = minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons)
    return res, res.x if res.success else None

In [1015]:
# Extracting expected returns to calculte Sharpe Maximum
mu = portfolio['RetE%'].values / 100

In [1016]:
res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
print("Max Sharpe:", portfolio_performance(w_max, mu, cov, riskfree)[2], "pesos:", w_max)
# print("Ret Max Sharpe:", w_max)

Max Sharpe: 1.0398665444743018 pesos: [0.00000000e+00 0.00000000e+00 8.95905413e-16 9.49294171e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.23255687e-03
 7.48069746e-02 3.76260433e-02 7.21492179e-02 1.55912959e-02
 6.01067010e-02 2.96800823e-02 1.32052995e-01 3.60840119e-16
 3.95376093e-02 0.00000000e+00 0.00000000e+00 7.80384662e-02
 3.57248641e-01 0.00000000e+00]


In [1017]:
#  Add Maximum Sharpe ticker weights to portfolio dataframe
portfolio['ShMaxE-W'] = w_max.round(3) * 100
# display(portfolio)

In [1018]:
# Add Maximum Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShMaxE-W'] = round(portfolio_performance(w_max, mu, cov, riskfree)[2], 3)
# display(portfolio)

In [1019]:
# Add Sharpe Maximum Return column and the value in the first line
portfolio['ShMaxRetE%'] = [portfolio_performance(w_max, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShMaxRetE%'] = portfolio['ShMaxRetE%'].round(1)
# display(portfolio)

In [1020]:
# Portfolio TARGET SHARPE calculation function (adjustado com uso de limitsport)
def weights_for_target_sharpe(mu, cov, target_sharpe, riskfree, limitsport, tol=1e-6, maxiter=1000):
    """
    Encontra pesos que aproximem um Sharpe alvo, usando limites individuais em limitsport.
    Minimiza o erro quadrático entre Sharpe(weights) e target_sharpe.
    """

    n = len(mu)

    # Construir bounds usando os limites informados pelo usuário (limitsport)
    bounds = tuple((0.0, limitsport[i]) for i in range(n))

    # Restrição: soma dos pesos = 1
    cons = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}]

    # Chute inicial
    x0 = np.ones(n) / n

    def sharpe_err_sq(w):
        _, _, s = portfolio_performance(w, mu, cov, riskfree)
        return (s - target_sharpe)**2

    opts = {'maxiter': maxiter, 'ftol': tol}
    res = minimize(sharpe_err_sq, x0, method='SLSQP', bounds=bounds, constraints=cons, options=opts)

    if res.success:
        ret, vol, s = portfolio_performance(res.x, mu, cov, riskfree)
        info = {'target_sharpe': target_sharpe, 'achieved_sharpe': s, 'ret': ret, 'vol': vol}
    else:
        info = {'message': res.message}

    return res, res.x if res.success else None, info

In [1021]:
# --- Defina aqui a sua lista original de limites em percentuais (exemplo que você forneceu) ---
'''limitsport_percent = [0.0, 0.0, 15.0, 15.0, 15.0, 15.0, 5.0, 10.0, 10.0,
                      10.0, 5.0, 10.0, 15.0, 5.0, 8.0, 10.0, 5.0, 5.0,
                      15.0, 15.0]  # exemplo: valores em % (0..100)'''

# Converta para fração (0..1)
limitsport = [float(x) / 100.0 for x in limitsassetsport]

# Verifique viabilidade: soma dos máximos deve ser >= 1 para poder ter soma(w)=1
sum_limits = sum(limitsport)
if sum_limits < 1.0 - 1e-12:
    # Estratégia: escalar limites proporcionalmente até que a soma seja 1.0
    # (mantendo cada limite <= 1.0). Isso preserva proporção entre limites.
    scale = 1.0 / sum_limits
    limitsport = [min(1.0, l * scale) for l in limitsport]
    print(f"AVISO: soma dos limites < 1.0 — limites escalados proporcionalmente (scale={scale:.6f}) para tornar o problema viável.")
    # recompute soma para informação
    print("Novos limites (fração):", limitsport)
    print("Soma dos novos limites:", sum(limitsport))

# Defina o target (seu código)
# Target como porcentagem do ShMaxE-W
target = portfolio.at[0, 'ShMaxE-W'] * target_per / 100
print("Target:", target)

# Chamada CORRETA: passe limitsport (em fração 0..1) como argumento
res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree, limitsport)

# Debug/saída (opcional)
# print("Resultado objetivo:", info)
# print("Pesos:", w_tgt)
# print("Achieved Sharpe:", info.get('achieved_sharpe'))


Target: 0.884


In [1022]:
#  Add Target Sharpe ticker weights to portfolio dataframe
portfolio['ShTg-W'] = (w_tgt * 100).round(3)
# Add Target (Achieved) Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShTg-W'] = round(info['achieved_sharpe'], 3)
# Add Target Maximum Return column and the value in the first line
portfolio['ShTgRetE%'] = [portfolio_performance(w_tgt, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShTgRetE%'] = portfolio['ShTgRetE%'].round(1)
display(portfolio)

,Ticker,W,RetE%,RetH%,Risk%,Beta,Min,Max,RetET%,RiskET%,SharpeE,RetHT%,SharpeH,ShMaxE-W,ShMaxRetE%,ShTg-W,ShTgRetE%
0,IBOV,0.000,0.0,11.0,14.0,0.359,118533.00,157749.00,20.0,9.0,0.654,19.0,0.553,1.04,21.9,0.823,21.9
1,USDBRL,0.000,8.0,4.9,12.0,0.003,4.81,6.30,0.0,0.0,0.000,0.0,0.000,0.00,0.0,0.000,0.0
2,AMZN,5.081,23.9,28.1,35.0,0.562,710.03,1420.95,0.0,0.0,0.000,0.0,0.000,0.00,0.0,6.335,0.0
3,BBSE3,7.509,18.5,12.4,17.0,0.412,25.33,40.42,0.0,0.0,0.000,0.0,0.000,9.50,0.0,9.099,0.0
4,BRK-B,10.812,18.5,23.3,21.0,0.312,1718.18,3135.39,0.0,0.0,0.000,0.0,0.000,0.00,0.0,9.099,0.0
5,CPFE3,0.000,2.7,14.6,19.0,0.605,31.00,47.77,0.0,0.0,0.000,0.0,0.000,0.00,0.0,0.000,0.0
6,ELET6,8.292,6.0,30.9,22.0,1.012,34.25,64.10,0.0,0.0,0.000,0.0,0.000,0.00,0.0,0.000,0.0
7,FBTC,2.714,25.9,43.9,52.0,0.556,171.30,583.82,0.0,0.0,0.000,0.0,0.000,0.80,0.0,3.132,0.0
8,HTMX11,5.311,20.5,4.8,28.0,0.028,119.99,177.06,0.0,0.0,0.000,0.0,0.000,7.50,0.0,9.099,0.0
9,JHSF3,0.000,25.3,39.0,33.0,1.443,3.33,7.29,0.0,0.0,0.000,0.0,0.000,3.80,0.0,4.549,0.0


In [1023]:
# Autentication in Google Docs (only once)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [1024]:
# Open workbook and worksheets
wb = gc.open('Quotes')
wss = wb.worksheet('Stockvar')
wsr = wb.worksheet('RealStatevar')
wssus = wb.worksheet('StockUSvar')
wseus = wb.worksheet('ETFUSvar')
wsrus = wb.worksheet('RealStateUSvar')
wsport = wb.worksheet('Portfolio')

In [1025]:
# Write data in the worksheets
wss.update([stockvar.columns.values.tolist()] + stockvar.values.tolist())
wsr.update([realstatevar.columns.values.tolist()] + realstatevar.values.tolist())
wssus.update([stockusvar.columns.values.tolist()] + stockusvar.values.tolist())
wseus.update([etfusvar.columns.values.tolist()] + etfusvar.values.tolist())
wsrus.update([reitusvar.columns.values.tolist()] + reitusvar.values.tolist())
wsport.update([portfolio.columns.values.tolist()] + portfolio.values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'Portfolio!A1:Q23',
 'updatedRows': 23,
 'updatedColumns': 17,
 'updatedCells': 391}

In [1026]:
# B3 REAL STATE FUNDS EXPLORER ROUTINE
#
# example tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MALL11.SA','MFII11.SA','SADI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']
# Remove ".SA" from real state funds dataframe tickers
tickerradjusted = [nome.replace('.SA', '') for nome in tickerr]
# Remove "IFIX" ticker
if "IFIX" in tickerradjusted:
    tickerradjusted.remove("IFIX")
#
# Funds list filter, exclude tickers not in real state funds list
fundsexplorer = fundsexplorer[fundsexplorer['Fundos'].isin(tickerradjusted)]
# display(fundsexplorer)
#
# Open worksheet
wsrfunds = wb.worksheet('FundsExplorer')
# Open worksheet
wsrfunds = wb.worksheet('FundsExplorer')

In [1027]:
# US STOCKS YAHOO FINANCE ROUTINE
#
# Example tickersus = ['^GSPC','USDBRL=X','AAPL','AIG','BAC','RIO','DHI','EXC','KMB','KO','LOPE','LYB','MGA','MSFT','MSTR','NUE','NVDA','TGT','TMUS','UPS','UNH','XOM']
#
# Dicionário solicitado para armazenar os dados (cada valor é um dict por ticker)
yfstockusdata = {}

# Conjunto para acumular todas as chaves encontradas em todos os tickers
all_keys = set()

for t in tickersus:
    try:
        tk = yf.Ticker(t)
        info = tk.info or {}
        # Garante que o ticker esteja presente como campo
        info_row = {'Ticker': t}
        # adiciona todas as chaves/valores retornados por .info
        for k, v in info.items():
            info_row[k] = v
            all_keys.add(k)
        # armazena a linha
        yfstockusdata[t] = info_row
    except Exception as e:
        # Em caso de falha, registra o ticker com apenas o campo 'Ticker'
        # e mantém as demais chaves ausentes (serão NaN no DataFrame)
        yfstockusdata[t] = {'Ticker': t}
        # (opcional) você pode logar o erro se quiser:
        # print(f"Erro em {t}: {e}")

# Para garantir colunas ordenadas com 'Ticker' primeiro, montamos explicitamente a lista de colunas
cols_other = sorted(all_keys - {'Ticker'})  # ordena alfabeticamente as demais chaves (opcional)
cols_final = ['Ticker'] + cols_other

# Converte o dicionário para DataFrame mantendo índice numérico (0,1,2,...)
# Note: usamos list(yfstockusdata.values()) para preservar cada dict como uma linha
yfstockus = pd.DataFrame(list(yfstockusdata.values()))

# Reindexa as colunas para garantir 'Ticker' como primeira e as restantes presentes
# Existem casos em que algumas chaves não aparecem em nenhum ticker — isso não ocorrerá,
# pois all_keys foi montado a partir de .info de cada ticker.
# Ainda assim, para segurança:
existing_cols = [c for c in cols_final if c in yfstockus.columns]
yfstockus = yfstockus[existing_cols + [c for c in yfstockus.columns if c not in existing_cols]]

# Exibe o DataFrame (uso em Jupyter/IPython)
# display(yfstockus)

In [1028]:
# Renaming ^GSPC to SP500 and USDBRLX to USDBRL
yfstockus['Ticker'] = yfstockus['Ticker'].str.replace('^GSPC', 'SP500', regex=False)
yfstockus['Ticker'] = yfstockus['Ticker'].str.replace('USDBRL=X', 'USDBRL', regex=False)
# display(yfstockus)

In [1029]:
# --- Função de sanitização por valor ---
def sanitize_value(v, maxlen=500):
    # None / NaN
    if v is None:
        return ''
    # floats: rejeitar NaN/Inf
    if isinstance(v, float):
        if not np.isfinite(v):
            return ''
        return float(v)
    # números inteiros e booleanos são OK
    if isinstance(v, (int, bool, np.integer, np.bool_)):
        return int(v) if isinstance(v, (int, np.integer)) else bool(v)
    # strings curtas OK (trunca se muito longas)
    if isinstance(v, str):
        return v if len(v) <= maxlen else v[:maxlen]
    # séries temporais / numpy types convertidos para string truncada
    # para todos os demais tipos (dict, list, Timestamp, ndarray, Decimal, etc.)
    try:
        s = str(v)
        return s if len(s) <= maxlen else s[:maxlen]
    except Exception:
        return ''

# --- Sanitizar coluna a coluna usando Series.map (evita applymap) ---
yfstockus_sanitized = yfstockus.copy()

for col in yfstockus_sanitized.columns:
    # Usa map para aplicar a função de maneira vetorizada por coluna
    yfstockus_sanitized[col] = yfstockus_sanitized[col].map(lambda x: sanitize_value(x))

# --- Monta as linhas para enviar ao Google Sheets ---
rows = [yfstockus_sanitized.columns.tolist()] + yfstockus_sanitized.values.tolist()

# --- Abre/Cria worksheet e atualiza ---
try:
    wssyfstockus = wb.worksheet('YFStockUS')
except Exception as e:
    # se não existir, cria (ajuste linhas/cols se quiser um tamanho específico)
    wssyfstockus = wb.add_worksheet(title='YFStockUS', rows=str(len(rows)+10), cols=str(len(rows[0])+5))

# Faz o update (agora com dados serializáveis)
wssyfstockus.update(rows)


{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'YFStockUS!A1:GF33',
 'updatedRows': 33,
 'updatedColumns': 188,
 'updatedCells': 6204}